In [1]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler

dataset = np.loadtxt("./features.txt", delimiter=" ");
print(dataset.shape)

X = dataset[:, :33]
Y = dataset[:, 34]
botnetDataset = dataset[dataset[:,34]==1, :]
normalDataset = dataset[dataset[:,34]==0, :]

botnetX = botnetDataset[:,:33]
botnetY = botnetDataset[:, 34]

normalX = normalDataset[:,:33]
normalY = normalDataset[:, 34]

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledBotnetX = scaler.fit_transform(botnetX)
rescaledNormalX = scaler.fit_transform(normalX)
rescaledX = scaler.fit_transform(X)

(52580, 35)


In [2]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)
model = model.fit(rescaledX, Y)

model.score(rescaledX, Y)

0.98721947508558383

In [3]:
def GetNextBotnetSet(rescaledX, Y):
    step = 4000
    prevStep = 0
    while step < rescaledX.shape[0]:
        yield rescaledX[prevStep:step, :], Y[prevStep:step]
        prevStep = step
        step = step + 4000
        
    return rescaledX[prevStep:, :], Y[prevStep:]

In [4]:
from sklearn.cross_validation import train_test_split

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model2 = RandomForestClassifier(n_estimators=100)
    model2 = model2.fit(X_train, Y_train)
    print(model2.score(X_test, Y_test))

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.999383096854
0.999383096854
1.0
0.998149290561
0.997532387415
0.997532387415
0.999383096854
0.999383096854
0.999383096854
0.998149290561
0.998766193708
0.998149290561


In [9]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    scores = cross_val_score(RandomForestClassifier(n_estimators=1), X, Y, scoring='accuracy', cv=10)
    predicted = cross_val_predict(RandomForestClassifier(n_estimators=1), X, Y, cv=10)
    print(scores.mean(), metrics.accuracy_score(Y, predicted))

0.993458921313 0.993210714727
0.993086571981 0.986297987903
0.987165669574 0.990248117516
0.993704312615 0.994692013332
0.989261694905 0.992223182323
0.99382746495 0.995802987286
0.995556164467 0.996296753487
0.991976374237 0.992099740773
0.994321596566 0.995926428836
0.988518822974 0.980249351932
0.988520954164 0.989260585113
0.986179537532 0.986174546352
